In [99]:
import tiktoken
from openai import OpenAI

tokenizer = tiktoken.get_encoding("cl100k_base")
client = OpenAI()

In [100]:
MODEL = "gpt-4o"

In [101]:
reducing_example0 = """Q: The median age in the city was 22.1 years. 10.1% of residents were under the age of 18; 56.2% were between the ages of 18 and 24; 16.1% were from 25 to 44; 10.5% were from 45 to 64; and 7% were 65 years of age or older. Which age group is larger: under the age of 18 or 18 and 24?
A: To answer the question "Which age group is larger: under the age of 18 or 18 and 24?", we need to know: "How many percent were under the age of 18?", "How many percent were between the ages of 18 and 24?"."""
reducing_example1 = """Q: Old age pensions were raised by 300 francs per month to 1,700 francs for a single person and to 3,700 francs for a couple, while health insurance benefits were made more widely available to unemployed persons and part-time employees. How many francs were the old age pensions for a single person before they were raised?
A: To answer the question "How many francs were the old age pensions for a single person before they were raised?", we need to know: "How many francs were the old age pensions for a single person?", "How many francs were old age pensions raised for a single person?"."""
reducing_example2 = """Q: In April 2011, the ECB raised interest rates for the first time since 2008 from 1% to 1.25%, with a further increase to 1.50% in July 2011. However, in 2012-2013 the ECB lowered interest rates to encourage economic growth, reaching the historically low 0.25% in November 2013. Soon after the rates were cut to 0.15%, then on 4 September 2014 the central bank reduced the rates from 0.15% to 0.05%, the lowest rates on record. How many percentage points did interest rates drop between April 2011 and September 2014?
A: To answer the question "How many percentage points did interest rates drop between April 2011 and September 2014?", we need to know: "What was the interest rate in April 2011?", "What was the interest rate in September 2014?"."""
reducing_example3 = """Q: Non-nationals make up more than half of the population of Bahrain. According to government statistics dated between 2005-2009 roughly 290,000 Indians, 125,000 Bangladeshis, 45,000 Pakistanis, 45,000 Filipinos, and 8,000 Indonesians. How many Pakistanis and Indonesians are in Bahrain?
A: To answer the question "How many Pakistanis and Indonesians are in Bahrain?", we need to know: "How many Pakistanis are in Bahrain?", "How many Indonesians are in Bahrain?"."""
reducing_example4 = """Q: The Census Bureaus 2006-2010 American Community Survey showed that (in 2010 inflation adjustment dollars) median household income was $52,056 and the median family income was $58,942. How many years did the Census Bureaus American Community Survey last?
A: To answer the question "How many years did the Census Bureaus American Community Survey last?", we need to know: "When did the Census Bureaus American Community Survey start?", "When did the Census Bureaus American Community Survey end?"."""

reducing_examples = "\n\n".join(
    [
        f"""Example {i}:
```
{example}
```"""
        for i, example in enumerate(
            [
                reducing_example0,
                reducing_example1,
                reducing_example2,
                reducing_example3,
                reducing_example4,
            ]
        )
    ]
)

In [102]:
reducing_system_message = f"""You are an AI assistant tasked with identifying the key pieces of information needed to answer a given question. Your goal is to break down the question into its essential components without actually answering it.

Here are some examples.

{reducing_examples}

(END OF EXAMPLES)

IMPORTANT: The broken down questions must be separated by commas.
"""

reducing_system_message_tokens = len(tokenizer.encode(reducing_system_message))
print("Tokens:", reducing_system_message_tokens)
print()
print(reducing_system_message)

Tokens: 826

You are an AI assistant tasked with identifying the key pieces of information needed to answer a given question. Your goal is to break down the question into its essential components without actually answering it.

Here are some examples.

Example 0:
```
Q: The median age in the city was 22.1 years. 10.1% of residents were under the age of 18; 56.2% were between the ages of 18 and 24; 16.1% were from 25 to 44; 10.5% were from 45 to 64; and 7% were 65 years of age or older. Which age group is larger: under the age of 18 or 18 and 24?
A: To answer the question "Which age group is larger: under the age of 18 or 18 and 24?", we need to know: "How many percent were under the age of 18?", "How many percent were between the ages of 18 and 24?".
```

Example 1:
```
Q: Old age pensions were raised by 300 francs per month to 1,700 francs for a single person and to 3,700 francs for a couple, while health insurance benefits were made more widely available to unemployed persons and par

In [103]:
solving_example0 = """The city of Bangkok has a population of 8,280,925. The census showed that it is home to 81,570 Japanese and 55,893 Chinese nationals, as well as 117,071 expatriates from other Asian countries, 48,341 from Europe, 23,418 from the Americas.
Q: How many Chinese nationals are there?
A: "The census showed that it is home to 81,570 Japanese and 55,893 Chinese nationals". So the answer is 55,893 Chinese nationals.
Q: How many European nationals are there?
A: "48,341 from Europe". So the answer is 48,341.
Q: How many more Chinese nationals are there than European nationals?
A: We know that there are 55,893 Chinese nationals. We also know that there are 48,341 European nationals. So there are 55,893 - 48,341 = 7552 more Chinese nationals than European nationals. So the answer is 7552."""
solving_example1 = """The gender distribution of the population was 50.2% male and 49.8% female. Of the adult population, 29 people or 14.6% of the population are between 20 and 29 years old. 28 people or 14.1% are 30 to 39, 36 people or 18.2% are 40 to 49, and 31 people or 15.7% are 50 to 59.
Q: What is the percentage of people between 40 to 49?.
A: "36 people or 18.2% are 40 to 49". So the answer is 18.2
Q: How many percent of people were not 40 to 49?
A: We know that 18.2% are 40 to 49. So 100% - 18.2% = 81.8% are not 40 to 49. So the answer is 81.8%."""
solving_example2 = """During the 1970s the oil shocks (1973 oil crisis and 1979 energy crisis) hit Iceland hard. Inflation rose to 43% in 1974 and 59% in 1980, falling to 15% in 1987 but rising to 30% in 1988. Iceland experienced moderately strong GDP growth (3% on average) from 1995 to 2004. Growth slowed between 2000 and 2002, but the economy expanded by 4.3% in 2003 and grew by 6.2% in 2004. Growth in 2005 exceeded 6%.
Q: When did the oil crisis occur?
A: "1973 oil crisis and 1979 energy crisis". So the answer is 1973.
Q: When did the energy crisis occur?
A: "1979 energy crisis". So the answer is 1979.
Q: How many years passed between the oil and energy crises?
A: We know that the oil crisis occurred in 1973. We also know that the energy crisis occurred in 1979. So 1979 - 1973 = 6 years passed between the oil and energy crises. So the answer is 6."""


solving_examples = "\n\n".join(
    [
        f"""Example {i}:
```
{example}
```"""
        for i, example in enumerate(
            [
                solving_example0,
                solving_example1,
                solving_example2,
            ]
        )
    ]
)

In [104]:
solving_system_message = f"""You are an AI assistant designed to answer questions.

Here are some examples.

{solving_examples}

(END OF EXAMPLES)
"""

solving_system_message_tokens = len(tokenizer.encode(solving_system_message))
print("Tokens:", solving_system_message_tokens)
print()
print(solving_system_message)

Tokens: 715

You are an AI assistant designed to answer questions.

Here are some examples.

Example 0:
```
The city of Bangkok has a population of 8,280,925. The census showed that it is home to 81,570 Japanese and 55,893 Chinese nationals, as well as 117,071 expatriates from other Asian countries, 48,341 from Europe, 23,418 from the Americas.
Q: How many Chinese nationals are there?
A: "The census showed that it is home to 81,570 Japanese and 55,893 Chinese nationals". So the answer is 55,893 Chinese nationals.
Q: How many European nationals are there?
A: "48,341 from Europe". So the answer is 48,341.
Q: How many more Chinese nationals are there than European nationals?
A: We know that there are 55,893 Chinese nationals. We also know that there are 48,341 European nationals. So there are 55,893 - 48,341 = 7552 more Chinese nationals than European nationals. So the answer is 7552.
```

Example 1:
```
The gender distribution of the population was 50.2% male and 49.8% female. Of the adu

In [105]:
example_question = 'The first two Buick automobiles were made in 1899 and 1900 at the "Buick Auto-Vim and Power Company" by chief-engineer Walter Lorenzo Marr, but company owner David Dunbar Buick was reluctant to begin making automobiles, being satisfied with stationary and marine engine production, so Marr left Buick in 1901 to found Marr (automobile) under his own name. His replacement was Eugene Richard, who applied for a patent in 1902 for Marrs valve-in-head (Overhead valve engine) engine, which patent, number 771,095, was awarded to Richard in the name of Buick in 1904. In 1903, the third Buick automobile was made, this time by Richard, but in 1904 Buick, whose company was now called "Buick Motor Company," moved to Flint, Michigan, and Richard stayed behind. Marr was rehired in Flint as chief engineer, to begin making automobiles in production. That year, 37 Buick automobiles were made, production increasing to 750 in 1905, 1,400 in 1906, 4,641 in 1907, and 8,800 in 1908, taking the number one spot away from close competitors Oldsmobile, Ford Motor Company, and Maxwell automobile. How many Buick automobiles were built between 1904 and 1905?'
example_solves = [
    (
        "How many Buick automobiles were built in 1904?",
        'In 1903, the third Buick automobile was made, this time by Richard, but in 1904 Buick, whose company was now called "Buick Motor Company," moved to Flint, Michigan, and Richard stayed behind. Marr was rehired in Flint as chief engineer, to begin making automobiles in production. That year, 37 Buick automobiles were made". So the answer is 37.',
    ),
    (
        "How many Buick automobiles were built in 1905?",
        'production increasing to 750 in 1905". So the answer is 750.',
    ),
]
example_reduce_response = 'To answer the question "How many Buick automobiles were built between 1904 and 1905?", we need to know: "How many Buick automobiles were built in 1904?", "How many Buick automobiles were built in 1905?", "How many Buick automobiles were built between 1904 and 1905?".'
example_subquestion = "How many Buick automobiles were built between 1904 and 1905?"

In [106]:
def reducing_user_prompt(question: str) -> str:
    return f"""Q: {question}
A: """


reducing_user_prompt_tokens = len(
    tokenizer.encode(reducing_user_prompt(example_question))
)
print("Tokens:", reducing_user_prompt_tokens)
print()
print(reducing_user_prompt(example_question))

Tokens: 291

Q: The first two Buick automobiles were made in 1899 and 1900 at the "Buick Auto-Vim and Power Company" by chief-engineer Walter Lorenzo Marr, but company owner David Dunbar Buick was reluctant to begin making automobiles, being satisfied with stationary and marine engine production, so Marr left Buick in 1901 to found Marr (automobile) under his own name. His replacement was Eugene Richard, who applied for a patent in 1902 for Marrs valve-in-head (Overhead valve engine) engine, which patent, number 771,095, was awarded to Richard in the name of Buick in 1904. In 1903, the third Buick automobile was made, this time by Richard, but in 1904 Buick, whose company was now called "Buick Motor Company," moved to Flint, Michigan, and Richard stayed behind. Marr was rehired in Flint as chief engineer, to begin making automobiles in production. That year, 37 Buick automobiles were made, production increasing to 750 in 1905, 1,400 in 1906, 4,641 in 1907, and 8,800 in 1908, taking the

In [107]:
def solving_user_prompt(
    question: str, solves: list[tuple[str, str]], subquestion: str
) -> str:
    solves_str = ""
    if len(solves) > 0:
        solves_str = "\n\n" + "\n".join(
            [f"Q: {solves[0]}\nA: {solves[1]}" for solves in solves]
        )

    return f"""{question}{solves_str}

Q: {subquestion}
A: """


solving_user_prompt_tokens = len(
    tokenizer.encode(
        solving_user_prompt(example_question, example_solves, example_subquestion)
    )
)
print("Tokens:", solving_user_prompt_tokens)
print()
print(solving_user_prompt(example_question, example_solves, example_subquestion))
print()
print(solving_user_prompt(example_question, [], example_subquestion))

Tokens: 440

The first two Buick automobiles were made in 1899 and 1900 at the "Buick Auto-Vim and Power Company" by chief-engineer Walter Lorenzo Marr, but company owner David Dunbar Buick was reluctant to begin making automobiles, being satisfied with stationary and marine engine production, so Marr left Buick in 1901 to found Marr (automobile) under his own name. His replacement was Eugene Richard, who applied for a patent in 1902 for Marrs valve-in-head (Overhead valve engine) engine, which patent, number 771,095, was awarded to Richard in the name of Buick in 1904. In 1903, the third Buick automobile was made, this time by Richard, but in 1904 Buick, whose company was now called "Buick Motor Company," moved to Flint, Michigan, and Richard stayed behind. Marr was rehired in Flint as chief engineer, to begin making automobiles in production. That year, 37 Buick automobiles were made, production increasing to 750 in 1905, 1,400 in 1906, 4,641 in 1907, and 8,800 in 1908, taking the nu

In [108]:
def decode_reduce_response(response: str) -> list[str]:
    # Extract the original question
    original_question = response.split('"')[1]

    # Split the response by the colon and take the second part
    questions_part = response.split(": ", 1)[1]

    # Remove the quotation marks and split by commas
    questions = [q.strip().strip('"') for q in questions_part.split('",')]

    # Remove the trailing period from the last question if it exists
    questions[-1] = questions[-1].rstrip(".")

    # Add the original question to the end of the list
    return questions + [original_question]


print(decode_reduce_response(example_reduce_response))

['How many Buick automobiles were built in 1904?', 'How many Buick automobiles were built in 1905?', 'How many Buick automobiles were built between 1904 and 1905?"', 'How many Buick automobiles were built between 1904 and 1905?']


In [109]:
def ask(question: str, debug=False) -> str:
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": reducing_system_message,
            },
            {
                "role": "user",
                "content": reducing_user_prompt(question),
            },
        ],
    )

    reducing_response = response.choices[0].message.content
    if debug:
        print("Reducing response:")
        print(reducing_response)
    subquestions = decode_reduce_response(reducing_response)

    if len(subquestions) == 0:
        raise ValueError("No subquestions found")

    solves: list[tuple[str, str]] = []
    for subquestion in subquestions:
        if debug:
            print("Subquestion:")
            print(subquestion)

        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {
                    "role": "system",
                    "content": solving_system_message,
                },
                {
                    "role": "user",
                    "content": solving_user_prompt(question, solves, subquestion),
                },
            ],
        )

        solving_response = response.choices[0].message.content
        if debug:
            print("Solving response:")
            print(solving_response)
        solves.append((subquestion, solving_response))

    answer = solves[-1][1]
    if debug:
        print("Answer:")
        print(answer)

    return answer

In [110]:
answer = ask(example_question, debug=True)

Reducing response:
To answer the question "How many Buick automobiles were built between 1904 and 1905?", we need to know: "How many Buick automobiles were built in 1904?", "How many Buick automobiles were built in 1905?".
Subquestion:
How many Buick automobiles were built in 1904?
Solving response:
In 1904, 37 Buick automobiles were made. So the answer is 37.
Subquestion:
How many Buick automobiles were built in 1905?"
Solving response:
In 1905, 750 Buick automobiles were made. So the answer is 750.
Subquestion:
How many Buick automobiles were built between 1904 and 1905?
Solving response:
To determine the total number of Buick automobiles built between 1904 and 1905, we need to add the number of automobiles made in both years.

- In 1904, 37 Buick automobiles were made.
- In 1905, 750 Buick automobiles were made.

So, the total number of Buick automobiles built between 1904 and 1905 is:

37 (in 1904) + 750 (in 1905) = 787.

So the answer is 787.
Answer:
To determine the total number 